# Data Cleaning Script

Use this script to create a cleaned csv file that will be used to interact with the home energy data visualization tool.

This is an internal script that will not be referenced at all in the final visualization tool code.

This purpose of this script is to take in 15 minute data CSV's from either Pecan Street or the ENEL samples and create one nice, cleaned CSV that will be uploaded to wherever the visualization tool can access it for use.

In [4]:
import pandas as pd
import numpy as np

# from bokeh.io import output_file
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
output_notebook()

Loading BokehJS ...

PART 2: Find our PV & EV homes with the energy use datasheets

In [5]:
#load in our data sets

original_TX_15min = pd.read_csv("C:/Users/Caleb/Desktop/Poolla Lab/Data/15minute_data_austin.csv.gz",compression='gzip')
original_CA_15min = pd.read_csv("C:/Users/Caleb/Desktop/Poolla Lab/Data/15minute_data_california.csv.gz",compression='gzip')
original_NY_15min = pd.read_csv("C:/Users/Caleb/Desktop/Poolla Lab/Data/15minute_data_newyork.csv.gz",compression='gzip')

#this is all the 15 min interval data available on PECAN STREET as of 3/26/2020



In [6]:
#add a state column to all thagg(ese and merge into one table to view
def give_state_col(df,state_str):
  df["state"]=state_str
  return df


edited_15min_CA=give_state_col(original_CA_15min,"CA")
edited_15min_NY=give_state_col(original_NY_15min,"NY")
edited_15min_TX=give_state_col(original_TX_15min,"TX")


#lets join all table into a master table for 15 min intervals
combined_15min=edited_15min_CA.append(edited_15min_NY,ignore_index=True)
combined_15min=combined_15min.append(edited_15min_TX,ignore_index=True)





In [7]:
#lets see how many homes are in this combined dataset and how many points at each home were taken total
home_overview=combined_15min.groupby("dataid").agg(len)
home_overview[["local_15min"]].rename(columns={"local_15min":"rows_data_available"})


,rows_data_available
dataid,
27,17663
142,17664
203,35036
387,17664
558,17664
...,...
9278,35035
9612,35036
9775,35132


In [10]:
#lets make a method to take in a Pecan St energy usage data file and give us only the data for homes with an EV and PV charger in a new df


def get_pv_ev_homes(raw_df,dataids_list):
  new_df=raw_df.loc[raw_df['dataid'].isin(dataids_list)]
  new_df

  return new_df


#get a df of EV and PV homes from the combined dataset 
#create a subset of this data that actually has PV and EV data
sorting_df2=combined_15min.groupby("dataid").agg(sum)
sorting_df2= sorting_df2.loc[(sorting_df2["car1"]!=0) & (sorting_df2["solar"]!=0),:]
final_PV_EV_dataids2=list(sorting_df2[["car1","solar"]].index)


In [11]:
all_states_15min_EV_PV_homes_only=get_pv_ev_homes(combined_15min,final_PV_EV_dataids2)
print("Homes with EV and PV 15min interval data available: \n")
all_states_15min_EV_PV_homes_only.groupby(["state","dataid"]).agg(len)[["air1"]].rename(columns={"air1":"rows_data_available"})

Homes with EV and PV 15min interval data available: 



rows_data_available
state dataid                     
NY    27                  17663.0
      1222                17664.0
      3000                17664.0
      5679                17664.0
TX    661                 35032.0
      1642                34648.0
      2335                34468.0
      4373                34532.0
      4767                34959.0
      6139                35036.0
      7719                34880.0
      8156                34984.0
      9278                35035.0

Now we have all 15 minute interval data from all 3 states together in one dataframe (combined_15min) and a subset of that data with only households that have both EV usage and PV generation (all_states_15min_EV_PV_homes_only)



# Now you can save the cleaned data from above at CSV files
Make sure only to run the following cells when you are ready to export your cleaned data as a CSV! IT takes a while to write!


In [ ]:
#make smaller table

In [12]:
# #write our new data to csv's
# all_states_15min_EV_PV_homes_only.to_csv("/content/drive/My Drive/Poolla Lab/CPS Project/Cleaned_Data/15min_EV_PV_homes_only.csv")
# combined_15min.to_csv("/content/drive/My Drive/Poolla Lab/CPS Project/Cleaned_Data/15min_all_homes_data.csv")


In [13]:
print(combined_15min.loc[:,"local_15min"].max())
print(combined_15min.loc[:,"local_15min"].min())

2019-10-31 23:45:00-05
2014-01-01 00:00:00-06
